In [1]:
import pandas as pd
import re
from sklearn.metrics import f1_score, recall_score, precision_score

In [2]:
def simple_search(rules, text):
    # Busca simples, usada principalmente em atos de pessoal
    return re.search(rules, text) != None

def long_search(rules, text):
    # Busca longa, usada em contrato e convênio
    principal, sub, titulo, low = rules
    
    if re.search(principal, text) == None:
        if re.search(sub, text) == None:
            if re.search(titulo, text) == None:
                if re.search(low, text.lower()) == None:
                    return False
    
    return True

def not_long_search(rules, text):
    # Busca maior que a simples mas menor que a longa, usada em aditamento
    principal, titulo, texto = rules
    
    if re.search(principal, text) == None:
        if re.search(titulo, text) == None:
            if re.search(texto, text) == None:
                return False
    
    return True

In [3]:
def get_scores(x, y):
    f1 = f1_score(x, y)
    recall_n = recall_score(x, y)
    precision_n = precision_score(x, y)
    
    return f1, recall_n, precision_n

In [4]:
# https://github.com/UnB-KnEDLe/DODFMiner/blob/classesNER/dodfminer/extract/polished/acts/licitacao.py
regex_licitacao = r'(?:AVISO\s+D[EO]\s+ABERTURA\s+D[EO]\s+LICITA[CÇ][AÃ]O|AVISO\s+ABERTURA\s+D[EO]\s+LICITA[CÇ][AÃ]O|AVISO\s+D[EO]\s+LICITA[CÇ][AÃ]O|AVISO\s+D[EO]\s+PREG[AÃ]O\s+ELETR[OÔ]NICO|AVISOS\s+D[EO]\s+ABERTURA\s+D[EO]\s+LICITA[CÇ][AÃ]O|AVISOS\s+D[EO]\s+LICITA[CÇ][AÃ]O|AVISOS\s+D[EO]\s+PREG[AÃ]O\s+ELETR[OÔ]NICO|AVISOS\s+D[EO]\s+ABERTURA\s+D[EO]\s+LICITA[CÇ][OÕ]ES|AVISOS?\s+D[EO]\s+LICITA[CÇ][OÕ]ES)'

In [5]:
# https://github.com/UnB-KnEDLe/DODFMiner/blob/classesNER/dodfminer/extract/polished/acts/anulacao_revogacao.py
regex_anulacao_revogacao = r'(?:AVISO\s+D[EO]\s+REVOGA[CÇ][AÃ]O\s+D[EO]\s+LICITA[CÇ][AÃ]O|AVISO\s+D[EO]\s+REVOGA[CÇ][AÃ]O|AVISO\s+D[EO]\s+ANULA[CÇ][AÃ]O\s+D[EO]\s+LICITA[CÇ][AÃ]O|AVISO\s+D[EO]\s+ANULA[CÇ][AÃ]O)'

In [6]:
# https://github.com/UnB-KnEDLe/DODFMiner/blob/classesNER/dodfminer/extract/polished/acts/suspensao.py
regex_suspensao = r'(?:AVISO\s+D[EO]\s+SUSPENS[AÃ]O\s+D[EO]\s+LICITA[CÇ][AÃ]O|AVISO\s+D[EO]\s+SUSPENS[AÃ]O)'

In [7]:
# https://github.com/UnB-KnEDLe/DODFMiner/blob/classesNER/dodfminer/extract/polished/acts/aditamento.py
principal_aditivo = r'(?:ADITIVO)'
regex_titulo_aditivo = r'(?:(ADITIVO[S]*\s.*CONTRAT[OUALIS]*)|(CONTRATO[S]*\s.*ADITIVO[S]*)|(ADITIVO,\s.*CONTRATO))'
regex_texto_aditivo = r'(?:(aditivo\sao\scontrato)|(espécie:\scontrato)|(termo\saditivo\s-\sao\scontrato))'

In [8]:
# https://github.com/UnB-KnEDLe/DODFMiner/blob/classesNER/dodfminer/extract/polished/acts/contrato_convenio.py
principal_contrato = r'(?:CONTRATO)'
regex_titulo_contrato = r'(?:(EXTRATO\sD[OE]\sCONTRATO)|(CONTRATO\sSIMPLIFICADO)|(CONTRATO\sPARA\sAQUISIÇÃO)|(^CONTRATO)'\
                        '|(EXTRATO\sD[OE]\sTERMO\sD[OE]\sCONTRATO\sNº\s)|(EXTRATO\sAO\sCONTRATO)|(CONTRATO\sDE\sPRESTAÇÃO\sDE\sSERVIÇOS\sNº)'\
                        '|(CONTRATO\sDE\sPATROCÍNIO)|(CONTRATO\sDE\sCONCESSÃO\sDE)|(^CONTRATO\sNº)|(^EXTRATOS\sDE\sCONTRATO[S]*))'
sub_contrato = r'(?:ADITIVO)'
low_contrato = 'termo aditivo ao contrato'

In [9]:
# https://github.com/UnB-KnEDLe/DODFMiner/blob/classesNER/dodfminer/extract/polished/acts/contrato_convenio.py
principal_convenio = r'(?:(CONVÊNIO)|(CONVENIO))'
regex_titulo_convenio = r'(?:(EXTRATO\sD[OE]\sCONVÊNIO)|(EXTRATO\sDE\sTERMO\sDE\sCONVÊNIO)|(CONVÊNIO\sSIMPLIFICADO)|(CONVÊNIO\sPARA\sAQUISIÇÃO)|(^CONVÊNIO)'\
                        '|(EXTRATO\sD[OE]\sTERMO\sD[OE]\sCONVÊNIO\sNº\s)|(EXTRATO\sAO\sCONVÊNIO)|(CONVÊNIO\sDE\sPRESTAÇÃO\sDE\sSERVIÇOS\sNº)'\
                        '|(CONVÊNIO\sDE\sPATROCÍNIO)|(CONVÊNIO\sDE\sCONCESSÃO\sDE)|(^CONVÊNIO\sNº)|(^EXTRATOS\sDE\sCONVÊNIO[S]*))'
sub_convenio = r'(?:ADITIVO)'
low_convenio = 'termo aditivo ao con'

In [10]:
# https://github.com/UnB-KnEDLe/DODFMiner/blob/classesNER/dodfminer/extract/polished/acts/abono.py
regex_abono = re.compile("(%s|%s|%s)" % (r"(Abono\sDE\sPERMANENCIA\s[(ao|equiva)][\s\S]*?)\s", r"([\s\S]*?", r"\d+\s*[\.|\-]\s*\d+\s*\/\s*\d+\s*\-\s*\d+)"))

In [11]:
# https://github.com/UnB-KnEDLe/DODFMiner/blob/classesNER/dodfminer/extract/polished/acts/cessoes.py
regex_cessoes = re.compile(r"([Pp][Rr][Oo][Cc][Ee][Ss][Ss][Oo][^0-9/]{0,12})([^\n]+?\n){0,2}?" + r"[^\n]*?[Aa]\s*[Ss]\s*[Ss]\s*[Uu]\s*[Nn]\s*[Tt]\s*[Oo]\s*:?\s*\bCESS.O\b" + r"([^\n]*\n){0,}?[^\n]*?(?=(?P<look_ahead>PROCESSO|Processo:|PUBLICAR|pertinentes[.]|autoridade cedente|" + r"(?i:publique-se)" + "))")

In [12]:
# https://github.com/UnB-KnEDLe/DODFMiner/blob/classesNER/dodfminer/extract/polished/acts/exoneracao.py
regex_exoneracao_naoefetivos = re.compile("(%s|%s|%s)" % (r"(EXONERAR)", r'((?=.*Comissao|.*\n.*Comissao|.*Especial|.*\n.*Especial )[\s\S]*?', r"(?:\.\n|NOMEAR|\d+\-\d+\/\d+\-\d+\.))"))
regex_exoneracao_efetivos = re.compile("(%s|%s|%s)" % (r"(EXONERAR,\s?)", r'((?:a\spedido,)?\s(?:[A-Z\\n\s]+),\s(?:matr[ií]cula\s(?:[0-9\.,X-])+)\s' + r'(?!.*\n?.*Cargo\sem\s+Comissao,|.*\n?.*Natureza\sEspecial,)[,\sa-zA-Z0-9\\\/-]*)', ""))

In [13]:
# https://github.com/UnB-KnEDLe/DODFMiner/blob/classesNER/dodfminer/extract/polished/acts/nomeacao.py
regex_nomeacao_comissionados = re.compile("(%s|%s|%s)" % (r"(NOMEAR)", r"([\s\S]*?)", r"((\.\s)|(?=(EXONERAR|NOMEAR)))"))
regex_nomeacao_efetivos = re.compile("(%s|%s|%s)" % (r"(NOMEAR\s)", r"((?:[ao]s\scandidat[ao]s\sabaixo(?:([a-zA-Z_0-9,\s\/-\:\-\(\);](no?\.)?)*).|" + r"(?:[ao]\scandidat[oa]\sabaixo(?:([a-zA-Z_0-9,\s\/-\:\-\(\);](no?\.)?)*)))(?:\s[a-zA-Z_\s]*" + r"(?:deficiencia|especiais):(?:\s[\sA-Zo]+,\s?\d{1,4}o?;?)+)?(?:\s)?(?:[\r\n\t\f\sa-zA-Z_\s]*" + r"classificacao:(?:\s[\sA-Zo]+,\s?\d{1,4}o?[,;]?)+)?)", ""))

In [14]:
# https://github.com/UnB-KnEDLe/DODFMiner/blob/classesNER/dodfminer/extract/polished/acts/reversoes.py
regex_reversoes = re.compile("(%s|%s)" % (r"(reverter\sa\satividade[,|\s])", r"(?<!lei)\s(?:[0-9|\s]*?[.|-]\s?)+?[0-9|\s]"))

In [15]:
# https://github.com/UnB-KnEDLe/DODFMiner/blob/classesNER/dodfminer/extract/polished/acts/substituicao.py
regex_substituicao = re.compile("(%s|%s|%s)" % (r"(DESIGNAR)", r"([\s\S]*?)", r"\.\s"))

In [16]:
# https://github.com/UnB-KnEDLe/DODFMiner/blob/classesNER/dodfminer/extract/polished/acts/sem_efeito_aposentadoria.py
regex_sem_efeito_aposentadoria = re.compile(r"TORNAR SEM EFEITO" + r"([^\n]+\n){0,10}?[^\n]*?" + r"(tempo\sde\sservi.o|aposentadoria|aposentou|([Dd][Ee][Ss])?[Aa][Vv][Ee][Rr][Bb][Aa]..[Oo]|(des)?averb(ar?|ou))" + r"[\d\D]{0,500}?[.]\s+" + r"(?=[A-Z]{3})")

In [17]:
data_contratos = pd.read_csv("DODFCorpus_contratos_licitacoes_v2.1.csv")

In [18]:
data_contratos = data_contratos.drop_duplicates(["texto_rel"])[["id_dodf", "tipo_rel", "texto_rel"]]

In [19]:
licitacao = data_contratos.apply(lambda row: simple_search(regex_licitacao, row["texto_rel"]), axis=1)
anulacao_revogacao = data_contratos.apply(lambda row: simple_search(regex_anulacao_revogacao, row["texto_rel"]), axis=1)
suspensao = data_contratos.apply(lambda row: simple_search(regex_suspensao, row["texto_rel"]), axis=1)

In [20]:
contrato = data_contratos.apply(lambda row: long_search([principal_contrato, regex_titulo_contrato, sub_contrato, low_contrato], row["texto_rel"]), axis=1)
convenio = data_contratos.apply(lambda row: long_search([principal_convenio, regex_titulo_convenio, sub_convenio, low_convenio], row["texto_rel"]), axis=1)

In [21]:
aditamento = data_contratos.apply(lambda row: not_long_search([principal_aditivo, regex_titulo_aditivo, regex_texto_aditivo], row["texto_rel"]), axis = 1)

In [22]:
data_pessoal = pd.read_csv("atos.csv")

In [23]:
abono = data_pessoal.apply(lambda row: simple_search(regex_abono, row["Texto"]), axis=1)
cessoes = data_pessoal.apply(lambda row: simple_search(regex_cessoes, row["Texto"]), axis=1)

exoneracao_naoefetivos = data_pessoal.apply(lambda row: simple_search(regex_exoneracao_naoefetivos, row["Texto"]), axis=1)
exoneracao_efetivos = data_pessoal.apply(lambda row: simple_search(regex_exoneracao_efetivos, row["Texto"]), axis=1)

nomeacao_comissionados = data_pessoal.apply(lambda row: simple_search(regex_nomeacao_comissionados, row["Texto"]), axis=1)
nomeacao_efetivos = data_pessoal.apply(lambda row: simple_search(regex_nomeacao_efetivos, row["Texto"]), axis=1)

reversoes = data_pessoal.apply(lambda row: simple_search(regex_reversoes, row["Texto"]), axis=1)
substituicao = data_pessoal.apply(lambda row: simple_search(regex_substituicao, row["Texto"]), axis=1)
sem_efeito_aposentadoria = data_pessoal.apply(lambda row: simple_search(regex_sem_efeito_aposentadoria, row["Texto"]), axis=1)

In [34]:
get_scores(data["tipo_rel"] == "AVISO_LICITACAO", licitacao)

(0.8443649373881931, 0.7386541471048513, 0.9853862212943633)

In [35]:
get_scores(data["tipo_rel"] == "AVISO_ANUL_REV_LICITACAO", anulacao_revogacao)

(0.9591836734693878, 0.9215686274509803, 1.0)

In [38]:
get_scores(data["tipo_rel"] == "AVISO_SUSPENSAO_LICITACAO", suspensao)

(0.9554140127388536, 0.9146341463414634, 1.0)

In [39]:
get_scores(data["tipo_rel"] == "EXTRATO_CONTRATO", contrato)

(0.665979381443299, 0.9584569732937686, 0.5102685624012638)

In [40]:
get_scores(data["tipo_rel"] == "EXTRATO_CONVENIO", convenio)

(0.03636363636363637, 0.90625, 0.018554062699936022)

In [41]:
get_scores(data["tipo_rel"] == "EXTRATO_ADITAMENTO_CONTRATUAL", aditamento)

(0.971086739780658, 0.945631067961165, 0.9979508196721312)

In [48]:
get_scores(data_pessoal["Tipo"] == "Ato_Abono_Permanencia", abono)

(0.029155787641427327, 1.0, 0.014793552660631487)

In [50]:
get_scores(data_pessoal["Tipo"] == "Ato_Cessao", cessoes)

(0.7149758454106281, 0.5584905660377358, 0.9932885906040269)

In [51]:
get_scores(data_pessoal["Tipo"] == "Ato_Exoneracao_Comissionado", exoneracao_naoefetivos)

(0.5276461295418642, 0.9975111996017919, 0.35868981564345803)

In [52]:
get_scores(data_pessoal["Tipo"] == "Ato_Exoneracao_Efetivo", exoneracao_efetivos)

(0.05183353048714916, 1.0, 0.026606314859792448)

In [53]:
get_scores(data_pessoal["Tipo"] == "Ato_Nomeacao_Comissionado", nomeacao_comissionados)

(0.4068243778031836, 1.0, 0.25535438286597484)

In [54]:
get_scores(data_pessoal["Tipo"] == "Ato_Nomeacao_Efetivo", nomeacao_efetivos)

(0.010105448154657295, 1.0, 0.005078383749172003)

In [55]:
get_scores(data_pessoal["Tipo"] == "Ato_Reversao", reversoes)

(0.01784340870635287, 1.0, 0.009002017693620985)

In [56]:
get_scores(data_pessoal["Tipo"] == "Ato_Substituicao", substituicao)

(0.4066842568161829, 1.0, 0.2552439832192537)

In [57]:
get_scores(data_pessoal["Tipo"] == "Ato_Tornado_Sem_Efeito_Apo", sem_efeito_aposentadoria)

/home/ciri/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.0, 0.0, 0.0)